In [1]:
# scRNA seq heart atlas
# Comparing expression in xenium panel, lineage level
# This is to make a list of genes to remove from expression analyses

In [2]:
import scanpy as sc
import pandas as pd

In [51]:
# Read in Global_lognormalised.h5ad file
adata = sc.read_h5ad('/scratch/aoill/projects/heart_transplant/scrna_atlas/Global_lognormalised.h5ad')

In [52]:
# Look at data
print(adata)

AnnData object with n_obs × n_vars = 704296 × 32732
    obs: 'sangerID', 'donor', 'donor_type', 'region', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'cell_type', 'cell_state', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score'
    var: 'gene_name_scRNA-0-original', 'gene_name_snRNA-1-original', 'gene_name_multiome-2-original', 'gene_id'
    uns: 'age_colors', 'cell_or_nuclei_colors', 'cell_state_colors', 'cell_type_colors', 'donor_colors', 'donor_type_colors', 'facility_colors', 'flushed_colors', 'gender_colors', 'kit_10x_colors', 'leiden', 'log1p', 'modality_colors', 'neighbors', 'original_or_new_colors', 'region_colors', 'region_finest_colors', 'scANVI_predictions_colors', 'umap'
    obsm: 'X_umap'
    obsp: 'connectivities', 'distances'


In [53]:
print(adata.obs.head())

                                         sangerID donor donor_type region  \
barcode                                                                     
HCAHeart7606896_GATGAGGCACGGCTAC  HCAHeart7606896    D1        DBD     AX   
HCAHeart7606896_CGCTTCACATTTGCCC  HCAHeart7606896    D1        DBD     AX   
HCAHeart7606896_GTTAAGCAGAGACTAT  HCAHeart7606896    D1        DBD     AX   
HCAHeart7606896_TCGCGTTGTAAGAGGA  HCAHeart7606896    D1        DBD     AX   
HCAHeart7606896_GCTGCGAGTGTTGGGA  HCAHeart7606896    D1        DBD     AX   

                                    age  gender facility cell_or_nuclei  \
barcode                                                                   
HCAHeart7606896_GATGAGGCACGGCTAC  50-55  Female   Sanger           Cell   
HCAHeart7606896_CGCTTCACATTTGCCC  50-55  Female   Sanger           Cell   
HCAHeart7606896_GTTAAGCAGAGACTAT  50-55  Female   Sanger           Cell   
HCAHeart7606896_TCGCGTTGTAAGAGGA  50-55  Female   Sanger           Cell   
HCAHeart76

In [54]:
print(adata.var.head())

              gene_name_scRNA-0-original gene_name_snRNA-1-original  \
gene_name-new                                                         
MIR1302-2HG                  MIR1302-2HG                MIR1302-2HG   
FAM138A                          FAM138A                    FAM138A   
OR4F5                              OR4F5                      OR4F5   
AL627309.1                    AL627309.1                 AL627309.1   
AL627309.3                    AL627309.3                 AL627309.3   

              gene_name_multiome-2-original          gene_id  
gene_name-new                                                 
MIR1302-2HG                     MIR1302-2HG  ENSG00000243485  
FAM138A                             FAM138A  ENSG00000237613  
OR4F5                                 OR4F5  ENSG00000186092  
AL627309.1                       AL627309.1  ENSG00000238009  
AL627309.3                       AL627309.3  ENSG00000239945  


In [55]:
# Is there only scRNA in this dataset?
unique_modalities = adata.obs['modality'].unique()
print(unique_modalities)
# 'Multiome-RNA', 'scRNA', 'snRNA'
# not sure if that matters
# no spatial so for now I'll just us all of the data

['scRNA', 'snRNA', 'Multiome-RNA']
Categories (3, object): ['Multiome-RNA', 'scRNA', 'snRNA']


In [56]:
unique_donors = adata.obs['donor'].unique()
print(unique_donors)
# there are 22 donors

['D1', 'D3', 'D4', 'D5', 'D6', ..., 'AV10', 'AV14', 'AV3', 'AV13', 'AH2']
Length: 22
Categories (22, object): ['A61', 'AH1', 'AH2', 'AV3', ..., 'H4', 'H5', 'H6', 'H7']


In [57]:
# What are the different regions of the heart sampled?
# Do we know in our data set and do we want to match where avaiable?
unique_regions = adata.obs['region'].unique()
print(unique_regions)
# 'SAN', 'AVN', 'RA', 'LA', 'RV', 'LV', 'SP', 'AX'

['AX', 'LV', 'RV', 'LA', 'SP', 'RA', 'SAN', 'AVN']
Categories (8, object): ['SAN', 'AVN', 'RA', 'LA', 'RV', 'LV', 'SP', 'AX']


In [58]:
# There is a column called cell_type in the metadata. I want 
# to see what are the cell types in this data set
unique_cell_types = adata.obs['cell_type'].unique()
print(unique_cell_types)

['Endothelial cell', 'Mural cell', 'Myeloid', 'Fibroblast', 'Lymphoid', ..., 'Lymphatic Endothelial cell', 'Mesothelial cell', 'Atrial Cardiomyocyte', 'Mast cell', 'Adipocyte']
Length: 12
Categories (12, object): ['Atrial Cardiomyocyte', 'Ventricular Cardiomyocyte', 'Fibroblast', 'Endothelial cell', ..., 'Adipocyte', 'Myeloid', 'Lymphoid', 'Mast cell']


In [59]:
#unique_cell_types = adata.obs['cell_state'].unique()
#for ct in unique_cell_types:
#    print(ct)

In [11]:
#for cell_type in unique_cell_types:
#    print(cell_type)

# Groupings to look at: Endothelial, myeloid, lymphoid, mesenchymal, and cardiomyocytes
#Endothelial cell - Endothelial 
#Mural cell - Mesenchymal
#Myeloid - Myeloid
#Fibroblast - Mesenchymal
#Lymphoid - Lymphoid
#Neural cell - Neural (not analyzing)
#Ventricular Cardiomyocyte - Cardiomyocyte
#Lymphatic Endothelial cell - Endothelial
#Mesothelial cell - Mesenchymal
#Atrial Cardiomyocyte - Cardiomyocyte
#Mast cell - Myeloid
#Adipocyte - Mesenchymal


In [60]:
# Add a column in the meta data called lineage
#cell_type_to_lineage = {
#    'Endothelial cell': 'Endothelial',
#    'Mural cell': 'Mesenchymal',
#    'Myeloid': 'Myeloid',
#    'Fibroblast': 'Mesenchymal',
#    'Lymphoid': 'Lymphoid',
#    'Neural cell': 'Neural',
#    'Ventricular Cardiomyocyte': 'Cardiomyocyte',
#    'Lymphatic Endothelial cell': 'Endothelial',
#    'Mesothelial cell': 'Mesenchymal',
#    'Atrial Cardiomyocyte': 'Cardiomyocyte',
#    'Mast cell': 'Myeloid',
#    'Adipocyte': 'Mesenchymal',
#}

# dont condense cell types that much
cell_type_to_lineage = {
    'Endothelial cell': 'Endothelial',
    'Mural cell': 'Mural',
    'Myeloid': 'Myeloid',
    'Fibroblast': 'Fibroblast',
    'Lymphoid': 'Lymphoid',
    'Neural cell': 'Neural',
    'Ventricular Cardiomyocyte': 'Cardiomyocyte',
    'Lymphatic Endothelial cell': 'Lymphatic_Endothelial',
    'Mesothelial cell': 'Mesothelial',
    'Atrial Cardiomyocyte': 'Cardiomyocyte',
    'Mast cell': 'Mast',
    'Adipocyte': 'Adipocyte',
}

# Add a new column 'lineage' based on the 'cell_type' column
adata.obs['lineage'] = adata.obs['cell_type'].map(cell_type_to_lineage)

# Verify the result
print(adata.obs[['cell_type', 'lineage']].head())

                                         cell_type      lineage
barcode                                                        
HCAHeart7606896_GATGAGGCACGGCTAC  Endothelial cell  Endothelial
HCAHeart7606896_CGCTTCACATTTGCCC        Mural cell        Mural
HCAHeart7606896_GTTAAGCAGAGACTAT  Endothelial cell  Endothelial
HCAHeart7606896_TCGCGTTGTAAGAGGA        Mural cell        Mural
HCAHeart7606896_GCTGCGAGTGTTGGGA  Endothelial cell  Endothelial


In [61]:
# Genes in my panel to analyze
xen_genes = [
    "ABCC11", "ABHD16A", "ACE2", "ACKR1", "ACTA2", "ACTG2", "ADAM28", "ADAMTS1", 
    "ADGRE1", "ADGRG7", "ADGRL4", "ADH1C", "ADH4", "ADIPOQ", "AGER", "AGR3", 
    "AHSP", "AICDA", "AIF1", "ALAS2", "ALB", "ALDH1A3", "AMY2A", "ANGPT2", 
    "ANKS1B", "ANPEP", "APCDD1", "APOA5", "APOBEC3A", "APOLD1", "AQP2", "AQP3", 
    "AQP8", "AQP9", "AR", "ARC", "ARFGEF3", "ASCL1", "ASCL3", "ASPN", "BAG6", 
    "BAMBI", "BANK1", "BASP1", "BBOX1", "BCL2L11", "BMX", "BRD2", "BTLA", "BTNL9", 
    "C15orf48", "C1orf162", "C1orf194", "C20orf85", "C4A", "C4B", "C5orf46", 
    "C6orf118", "C7", "CA4", "CAPN8", "CAV1", "CAVIN1", "CAVIN2", "CCDC39", 
    "CCDC78", "CCL19", "CCL27", "CCL4", "CCL5", "CCNB2", "CCR2", "CCR6", "CCR7", 
    "CCR8", "CD14", "CD163", "CD19", "CD1A", "CD1C", "CD1E", "CD2", "CD209", 
    "CD247", "CD27", "CD274", "CD28", "CD300E", "CD34", "CD3D", "CD3E", "CD4", 
    "CD40", "CD40LG", "CD5", "CD5L", "CD68", "CD69", "CD70", "CD72", "CD79A", 
    "CD80", "CD83", "CD86", "CD8A", "CD93", "CDH16", "CDH4", "CDK1", "CELF4", 
    "CENPF", "CFAP53", "CFB", "CFHR1", "CFHR3", "CFTR", "CHGA", "CHST15", 
    "CLC", "CLCA1", "CLCA2", "CLEC10A", "CLEC14A", "CLEC4E", "CLECL1", "CLIC6", 
    "CLNK", "CNN1", "COCH", "COL17A1", "COL1A1", "COL1A2", "COL22A1", "COL5A2", 
    "CPA3", "CRHBP", "CRISPLD2", "CSF2RA", "CSF2RB", "CSF3", "CTLA4", "CTSG", 
    "CTSK", "CTSS", "CX3CL1", "CX3CR1", "CXCL10", "CXCL11", "CXCL2", "CXCL6", 
    "CXCL8", "CXCL9", "CXCR3", "CXCR4", "CYP1A1", "CYP2A7", "CYP2B6", "CYP2F1", 
    "CYP3A4", "CYP4B1", "CYTIP", "DCN", "DERL3", "DES", "DIRAS3", "DMBT1", "DNAAF1", 
    "DNASE1L3", "DPEP1", "DPT", "DST", "DUSP2", "ECSCR", "EDN1", "EDNRB", 
    "EGFL7", "EGFR", "EHF", "ELF5", "EPCAM", "ERBB2", "ERG", "ESR1", "FAP", 
    "FAS", "FBLN1", "FBN1", "FCER1A", "FCER1G", "FCGR1A", "FCGR3A", "FCN1", 
    "FCN2", "FFAR2", "FGFBP1", "FGFBP2", "FGL2", "FHL2", "FKBP11", "FKBP1A", 
    "FLOT1", "FOSB", "FOXA1", "FOXI1", "FOXJ1", "FOXP3", "FSTL3", "FXYD2", 
    "GATA2", "GATA3", "GATM", "GCG", "GDF15", "GEM", "GGH", "GHRL", "GKN2", 
    "GLIPR1", "GLYATL1", "GNG11", "GNL1", "GNLY", "GPC1", "GPC3", "GPR183", 
    "GPRC5A", "GPX2", "GYPA", "GYPB", "GZMA", "GZMB", "GZMK", "HAMP", 
    "HAVCR2", "HEMGN", "HEPACAM2", "HES4", "HIGD1B", "HLA-DQB2", "HMGCS2", 
    "HPGDS", "HPX", "HS6ST3", "ICAM1", "IGF1", "IGSF6", "IL1B", "IL1R2", 
    "IL1RL1", "IL2RA", "IL3RA", "IL6R", "IL7R", "INMT", "INS", "IRF8", "ITGAM", 
    "ITGAX", "KCNH7", "KCNK3", "KCNMA1", "KIT", "KLK11", "KLRB1", "KLRC1", 
    "KLRD1", "KNG1", "KRT20", "KRT7", "LAG3", "LAMP3", "LEF1", "LGI4", "LGR5", 
    "LHFPL5", "LIF", "LILRA4", "LILRA5", "LILRB2", "LILRB4", "LPL", "LRTM1", 
    "LST1", "LTBP2", "LY6D", "LY6G5B", "LY86", "LYPD5", "LYVE1", "LYZ", 
    "MALL", "MAMDC2", "MARCO", "MCEMP1", "MCF2L", "MDM2", "MEDAG", "MEF2C", 
    "MEST", "MET", "MFAP5", "MINAR2", "MKI67", "MLANA", "MLPH", "MMRN1", 
    "MMRN2", "MNDA", "MPEG1", "MRC1", "MS4A1", "MS4A2", "MS4A3", "MS4A4A", 
    "MS4A6A", "MS4A7", "MSX1", "MTOR", "MTRNR2L11", "MYBPC1", "MYC", "MYD88", 
    "MYH11", "MYLK", "MZB1", "NAT8", "NKG7", "NOMO1", "NPDC1", "NTN4", "OGN", 
    "OPRPN", "PCNA", "PCOLCE", "PCP4", "PCSK2", "PDCD1", "PDGFRA", "PDGFRB", 
    "PDPN", "PEBP4", "PECAM1", "PGR", "PILRA", "PITX1", "PLA1A", "PLA2G2A", 
    "PLA2G2D", "PLA2G7", "PLAC9", "PLAUR", "PLCG2", "PLD4", "PLIN4", "PMP22", 
    "POSTN", "PPARG", "PPM1N", "PPP1R11", "PPP1R1A", "PPP1R1B", "PPY", 
    "PRCP", "PRDM1", "PRF1", "PRG4", "PROX1", "PSG2", "PSMB8", "PTGDS", "PTN", 
    "PTPRC", "PVALB", "RAMP2", "RAPGEF3", "RBP5", "RERGL", "RETN", "RGS16", 
    "RIDA", "RND1", "ROBO4", "RTKN2", "S100A1", "S100A12", "SCGB2A1", "SCGN", 
    "SCUBE1", "SELE", "SELL", "SELP", "SEMA3C", "SERPINB2", "SERPINB3", 
    "SERPINB9", "SFRP2", "SFRP4", "SFTA2", "SH2D3C", "SIGLEC1", "SLAMF1", 
    "SLAMF7", "SLC18A2", "SLC22A8", "SLC22A9", "SLC26A2", "SLC26A3", "SLC4A1", 
    "SMIM24", "SMYD2", "SNAI1", "SNCA", "SNCG", "SNTN", "SOX17", "SOX18", "SOX2", 
    "SPDEF", "SPI1", "SPIB", "SPP1", "SRPX", "SST", "STC1", "STC2", "STEAP4", "TAC1", 
    "TAP1", "TAT", "TBX3", "TCF15", "TCF4", "TCIM", "TCL1A", "TENT5C", "TFF2", "TFPI", 
    "THAP2", "THBS2", "THBS4", "THEMIS2", "THY1", "TIFAB", "TIMP4", "TLR4", "TM4SF18", 
    "TM4SF4", "TMC5", "TMEM100", "TMEM174", "TMEM52B", "TNC", "TNFRSF13B", "TNFRSF17", 
    "TNFRSF4", "TNFRSF9", "TNXB", "TOP2A", "TOX", "TRAC", "TREM2", "TRIM26", "TSPAN19", 
    "TSPY3", "TTN", "TYROBP", "UBD", "UBE2C", "UMOD", "UPK3B", "VCAM1", "VCAN", "VSIG4", 
    "VWA5A", "VWF", "XCR1", "ZCCHC12"
]

In [50]:
#genes_to_keep = []
#for lineage in adata.obs['lineage'].unique():
#    print(f"Processing lineage: {lineage}")
#    
#    # subset adata to just have the cells from the lineage being analyzed
#    lineage_adata = adata[adata.obs['lineage'] == lineage, :]
#    for gene in xen_genes:
#        #print(gene)
#        # get the counts for that gene across all the cells in the lineage
#        gene_index = lineage_adata.var_names.get_loc(gene)
#        sum_counts = lineage_adata.X[:, gene_index].sum()
#        # if the sum of those counts are greater than 0 then add that gene and cell type pair to a list
#        if sum_counts > 0:
#            genes_to_keep.append([lineage, gene])
#        
## Create a pandas DataFrame from the collected data
#df_genes_to_keep = pd.DataFrame(genes_to_keep, columns=['Lineage', 'Gene'])


In [62]:
# Lots of genes remained so maybe do has to have expression in at least 20% of cells?
# Make the table be lineage, gene, pt of cells with any expression
genes_to_keep = []
for lineage in adata.obs['lineage'].unique():
    print(f"Processing lineage: {lineage}")
    
    # subset adata to just have the cells from the lineage being analyzed
    lineage_adata = adata[adata.obs['lineage'] == lineage, :]
    for gene in xen_genes:
        #print(gene)
        # get the counts for that gene across all the cells in the lineage
        gene_expression_values = lineage_adata[:, gene].X.toarray().flatten()
        # get the number of cells with any expression
        cells_expressing_gene = (gene_expression_values > 0).sum()
        # get the total number of cells in the lineage
        total_cells_in_lineage = lineage_adata.n_obs
        if total_cells_in_lineage > 0:
            proportion_expressed = cells_expressing_gene / total_cells_in_lineage
            genes_to_keep.append([lineage, gene, proportion_expressed])

        
# Create a pandas DataFrame from the collected data
df_genes_to_keep = pd.DataFrame(genes_to_keep, columns=['Lineage', 'Gene', 'Proprtion_Expressed'])

Processing lineage: Endothelial
Processing lineage: Mural
Processing lineage: Myeloid
Processing lineage: Fibroblast
Processing lineage: Lymphoid
Processing lineage: Neural
Processing lineage: Cardiomyocyte
Processing lineage: Lymphatic_Endothelial
Processing lineage: Mesothelial
Processing lineage: Mast
Processing lineage: Adipocyte


In [40]:
# Testing code
#lineage_adata = adata[adata.obs['lineage'] == "Endothelial", :]
#gene_index = lineage_adata.var_names.get_loc("ABCC11")
#sum_counts = lineage_adata.X[:, gene_index].sum()
##sum_counts

##if sum_counts > 0:
##    print(sum_counts)

In [37]:
#adata.X
#adata.X[120:150, 100:110].toarray()

In [63]:
# Print the resulting DataFrame
print(df_genes_to_keep)

          Lineage     Gene  Proprtion_Expressed
0     Endothelial   ABCC11             0.001217
1     Endothelial  ABHD16A             0.011148
2     Endothelial     ACE2             0.004129
3     Endothelial    ACKR1             0.073594
4     Endothelial    ACTA2             0.048759
...           ...      ...                  ...
5242    Adipocyte    VSIG4             0.008666
5243    Adipocyte    VWA5A             0.028360
5244    Adipocyte      VWF             0.070112
5245    Adipocyte     XCR1             0.001103
5246    Adipocyte  ZCCHC12             0.000473

[5247 rows x 3 columns]


In [64]:
# Save the DataFrame to a CSV file
#df_genes_to_keep.to_csv('/scratch/aoill/projects/heart_transplant/scrna_atlas/genes_to_keep.csv', index=False)
df_genes_to_keep.to_csv('/scratch/aoill/projects/heart_transplant/scrna_atlas/genes_to_keep_proportions_new_groups.csv', index=False)